In [8]:
import import_ipynb
import cv2
import matplotlib.pyplot as plt
import numpy as np
from search import *

data = search('./binary_numbers')
data = data[1:]

fname = 'digits.npz'


# digits.png 를 한글자씩 배열로 잘라줌
def machineLearning():
    image = cv2.imread('digits.png')
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]
    x = np.array(cells)
    train = x[:,:].reshape(-1,400).astype(np.float32)

    k = np.arange(10)
    train_labels = np.repeat(k,500)[:,np.newaxis]

    np.savez(fname,train=train,train_labels = train_labels)

# digits.png 와 비교하기 위해 전처리
def resize20(pimg):
    img = cv2.imread(pimg)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    grayResize = cv2.resize(gray,(20,20))
    ret, thresh = cv2.threshold(grayResize, 150, 255,cv2.THRESH_BINARY)
#     plt.imshow(thresh, 'gray')
    return thresh.reshape(-1,400).astype(np.float32)


# digits.npz 파일을 불러오기
def loadTrainData(fname):
    with np.load(fname) as data:
        train = data['train']
        train_labels = data['train_labels']

    return train, train_labels


# digits.npz를 토대로 숫자 인식하기
def checkDigit(test, train, train_labels):
    knn = cv2.ml.KNearest_create()
    knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)

    ret, result, neighbours, dist = knn.findNearest(test, k=5)

    return result

# 숫자 인식 결과를 반환 해줌
def recognition(img):
    loadTrainData(fname)
    train, train_labels = loadTrainData(fname)
    img = resize20(img)
    result = checkDigit(img, train, train_labels)
    rec_res = result[0][0]
    return str(int(rec_res))


# 학습 데이터가 있을 경우 데이터를 하나씩 수기 입력하여 학습시키기
def traing_data(data):
    cnt = 0
    
    train, train_labels = loadTrainData(fname)
    
    for link in data:
        image = cv2.imread(link)

        # pc에서만 가능
    #     cv2.imshow('image', image)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()

        test = resize20(link)

        result = checkDigit(test, train, train_labels)
        print(int(result[0][0]), end = ' : ')

        user_input = input()
        if user_input == '':
            cnt += 1
            continue

        user_input = int(user_input)
        if user_input != int(result[0][0]):
            train = np.append(train, test, axis = 0)
            newLabel = np.array(user_input).reshape(-1,1)
            train_labels = np.append(train_labels, newLabel, axis = 0)
            np.savez(fname, train = train, train_labels = train_labels)
            print('SAVE SUCCESS')
            cnt -= 1

        cnt += 1

    print(f"{len(data)} : {cnt}")
    